# Whale Activity effect on Price Movement

In [1]:
import requests

coin_id = "ethereum"
url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
params = {
    "vs_currency": "usd",
    "days": "30"
}
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, params=params)
print(response.json())

{'prices': [[1752256908975, 2992.624183217603], [1752260638110, 2994.9467067038017], [1752264200864, 3002.772129714876], [1752267832698, 2982.624063201952], [1752271414360, 2964.5182915150644], [1752275010879, 2949.4584545678536], [1752278636142, 2958.8514450990992], [1752282234609, 2946.5968355648433], [1752285830345, 2961.1235436164607], [1752289429004, 2958.8709433316562], [1752293032192, 2963.61330941745], [1752296618650, 2953.5429183299552], [1752300201086, 2955.4093969651144], [1752303838125, 2961.875723053227], [1752307448480, 2964.6487227418274], [1752311006897, 2965.1048310608453], [1752314645792, 2974.5015985668506], [1752318206990, 2976.6456369036705], [1752321830698, 2967.511256690298], [1752325435099, 2937.877205699426], [1752329007725, 2936.477207543277], [1752332643253, 2932.518055493346], [1752336232028, 2912.956320971995], [1752339817095, 2926.8225229271793], [1752343435375, 2940.9948540256487], [1752346839689, 2940.9536184309977], [1752350607753, 2940.3660994311117], 

In [2]:



import os
import requests
import pandas as pd
from dotenv import load_dotenv



In [3]:
# Load API key from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
   

In [4]:
from dotenv import load_dotenv
load_dotenv()
import os
#DUNE_API_KEY = os.getenv("DUNE_API_KEY")
print(DUNE_API_KEY)  # This should print your actual API key (or part of it)

None


In [6]:
def get_whale_data(query_id):
    url = f"https://api.dune.com/api/v1/query/{query_id}/execute"
    headers = {"X-Dune-API-Key": DUNE_API_KEY}
    r = requests.post(url, headers=headers)
    print("Dune API response:", r.status_code, r.text)  # Debug print
    if "execution_id" not in r.json():
        raise Exception("No execution_id in response. Check your query_id and API key.")
    execution_id = r.json()["execution_id"]

    # Wait for the query to finish (simple polling)
    import time
    status_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"
    while True:
        status_resp = requests.get(status_url, headers=headers)
        status = status_resp.json().get("state")
        if status == "QUERY_STATE_COMPLETED":
            break
        elif status == "QUERY_STATE_FAILED":
            raise Exception("Dune query failed.")
        time.sleep(2)

    results_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results"
    r = requests.get(results_url, headers=headers)
    data = r.json()["result"]["rows"]
    df = pd.DataFrame(data)
    if "block_time" in df.columns:
        df["timestamp"] = pd.to_datetime(df["block_time"])
    return df

In [7]:
ethereum_whale_query_id = 5612037
eth_whale_df = get_whale_data(ethereum_whale_query_id)
print(eth_whale_df.head())

Dune API response: 401 {"error":"invalid API Key"}


Exception: No execution_id in response. Check your query_id and API key.